In [ ]:
!pip -q install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 108.3 MB/s eta 0:00:00


https://huggingface.co/HuggingFaceTB/SmolLM3-3B

In [ ]:
!nvidia-smi

Wed Jul  9 01:25:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----


## HuggingFaceTB/SmolLM3-3B



In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:

device = "cuda"  # for GPU usage or "cpu" for CPU usage
torch.set_default_device(device)

In [ ]:

model_name = "HuggingFaceTB/SmolLM3-3B"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
).to(device)

tokenizer = AutoTokenizer.from_pretrained(model_name,torch_dtype="auto")

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/182 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

## With Reasoning

In [ ]:
# prepare the model input
prompt = "Give me a brief explanation of gravity in simple terms."
messages_think = [
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages_think,
    tokenize=False,
    add_generation_prompt=True,
)


In [ ]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# Generate the output
generated_ids = model.generate(**model_inputs, max_new_tokens=32768)

# Get and decode the output
output_ids = generated_ids[0][len(model_inputs.input_ids[0]) :]

print(tokenizer.decode(output_ids, skip_special_tokens=True))

<think>
Okay, so I need to explain gravity in simple terms. Let me start by recalling what I know. Gravity is the force that pulls objects towards each other. I remember that the Earth pulls things towards it, which is why we don't float away. But how exactly does that work?

Hmm, Newton's law of universal gravitation comes to mind. He said that every mass attracts every other mass with a force proportional to their masses and inversely proportional to the square of the distance between them. So, the bigger the masses, the stronger the pull, and the farther apart they are, the weaker the force. That makes sense. But maybe I should simplify that a bit.

Wait, maybe I should start with an example. Like, why do we stay on the ground? Because the Earth's gravity is pulling us down. But why doesn't the Sun just fly away? Because it's also being pulled by the Earth. So it's a mutual force. But how does that work when we're on the surface of the Earth? The acceleration due to gravity is about

## With No Reasoning

In [ ]:
prompt = "Give me a brief explanation of gravity in simple terms."
messages = [
    {"role": "system", "content": "/no_think"},
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)

model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# Generate the output
generated_ids = model.generate(**model_inputs, max_new_tokens=32768)

# Get and decode the output
output_ids = generated_ids[0][len(model_inputs.input_ids[0]) :]
print(tokenizer.decode(output_ids, skip_special_tokens=True))

Gravity is a force that pulls objects towards each other. Imagine you're playing catch with a friend. When you throw a ball, it falls towards the ground because of gravity. Gravity is what makes things fall, and it's also what keeps the Earth and other planets in orbit around the sun. It's like a big invisible rope that connects everything in the universe, pulling them together. This force is always there, even when we can't see it. It's what makes the moon orbit the Earth, and it's what keeps us on the ground!


## Wrapping it all

In [ ]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [ ]:
def generate(input_text, system_prompt="Think carefully and with numbered thinking",max_length=512,enable_thinking=True):

    messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": input_text}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # conduct text completion
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_length
    )
    # decoded = tokenizer.batch_decode(generated_ids)
    # print(decoded[0])

    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

    # # parsing thinking content
    # try:
    #     # rindex finding 151668 (</think>)
    #     index = len(output_ids) - output_ids[::-1].index(151668)
    # except ValueError:
    #     index = 0

    # thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
    # content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

    # print("thinking content:", thinking_content)
    # print("content:", content)

    content = tokenizer.decode(output_ids, skip_special_tokens=True)
    wrapped_text = wrap_text(content)
    print(wrapped_text)

In [ ]:
generate('Write a detailed analogy between mathematics and a lighthouse.',
         max_length=32000,
         )

<think>
Okay, I need to write a detailed analogy between mathematics and a lighthouse. Let me
start by recalling what a lighthouse does. A lighthouse is a tall tower that emits a beam
of light to guide ships safely into harbor. It uses a rotating beam to cast light in
different directions, helping ships navigate through treacherous waters.

Now, how can I relate this to mathematics? Well, mathematics provides a system to
understand and navigate the world, just like a lighthouse guides ships. But I need to
think of specific aspects. Maybe the lighthouse's beam is like mathematical concepts that
help us understand patterns or navigate through complex ideas.

First, the lighthouse's purpose is to provide guidance. In mathematics, theorems and
formulas serve as guides for solving problems. So, the lighthouse's beam is like
mathematical truths or principles that help us find the right path.

The lighthouse uses a rotating beam to cover a wide area. Similarly, mathematics often
involves expl

## CodeGen

In [ ]:
generate('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''',
         max_length=32000,
         )

<think>

</think>
To implement the function `print_prime(n)` that prints all prime numbers between 1 and
`n`, we can follow these steps:

1.  **Define the Function:** The function `print_prime(n)` takes an integer `n` as input.
2.  **Initialize a List:** Create an empty list `primes` to store the prime numbers.
3.  **Iterate Through Numbers:** Loop through numbers from 2 to `n` (since 1 is not a
prime number).
4.  **Check for Primality:** For each number `i` in the range, check if it's a prime
number. To check for primality, iterate from 2 to the square root of `i` and check if `i`
is divisible by any of these numbers.
5.  **Add Prime to List:** If `i` is prime, add it to the `primes` list.
6.  **Print Primes:** After the loop, print all the prime numbers in the `primes` list.

Here's the Python code implementing these steps:

```python
import math

def print_prime(n):
    """
    Print all primes between 1 and n
    """
    primes = []
    for i in range(2, n + 1):
        is_prime = 

## Instruction Answering

In [ ]:
%%time

generate('Write a detailed breakdown of building a successful lemonade stand.',
         max_length=32000,
         )

<think>
Okay, the user wants a detailed breakdown of building a successful lemonade stand. Let me
start by breaking down the process step by step. First, I need to consider the basics like
location and materials. Where should the stand be placed? Maybe a busy area with foot
traffic. Then, what materials are needed? Lemonade ingredients, cups, umbrellas, maybe a
table or stand.

Next, the business plan. How much should they charge? Competitor pricing is important
here. Maybe start with a low price to attract customers and then adjust as the business
grows. Marketing is another key area. How will they attract customers? Flyers, social
media, word of mouth.

Operational aspects can't be overlooked. How will they make the lemonade? A simple recipe
with fresh lemons, sugar, water. Maybe a recipe card for consistency. Pricing and profit
margins need to be calculated. How many cups can they sell in an hour? That affects how
much they need to produce and how much they can make per hour.

Custo

In [ ]:
%%time

generate('Write a detailed breakdown of building a successful lemonade stand.',
         system_prompt="/no_think",
         max_length=32000,
         )

Building a successful lemonade stand requires careful planning, creativity, and execution.
Here's a detailed breakdown of the steps to take to create a lemonade stand that attracts
and retains customers:

**Step 1: Market Research**

Before setting up the stand, research the local market to understand the demand for
lemonade and the competition. Identify your target audience, such as children, teenagers,
and adults, and determine their preferences for lemonade flavors and prices. Look for
existing lemonade stands in the area to assess their strengths and weaknesses.

**Step 2: Choose a Location**

Select a location that is easily accessible and visible to potential customers. Consider
high-traffic areas such as parks, beaches, or near schools. Ensure that the location is
clean, well-lit, and has adequate seating for customers. Avoid areas with a lot of foot
traffic, as it can be overwhelming for customers and increase noise levels.

**Step 3: Design Your Stand**

Create a visually appe

In [ ]:
%%time

generate('What is the difference between a Llama, Vicuna and an Alpaca?',
         max_length=32000,
         )

<think>

</think>
Let's break down the differences between Llama, Vicuna, and Alpaca step by step:

1. **Species and Taxonomy**:
   - Llama (Lama glama) and Alpaca (Vicugna pacos) are both domesticated members of the
Camelidae family, which includes camels, llamas, alpacas, and vicuñas. Vicuña (Vicugna
vicugna) is a wild species, while the other two are domesticated.
   - Vicuña is considered a wild animal, whereas Llama and Alpaca are domesticated and
have been bred for thousands of years.

2. **Physical Characteristics**:
   - **Llama**: Llamas are larger than alpacas, with a height of up to 4 feet (1.2 meters)
at the shoulder and a weight of up to 300 pounds (136 kg). They have long, coarse hair
that can be shorn, and their ears are more prominent compared to alpacas.
   - **Alpaca**: Alpacas are smaller than llamas, with a height of around 3 feet (0.9
meters) at the shoulder and a weight of up to 175 pounds (80 kg). Their hair is softer and
finer than that of llamas, and it is high

In [ ]:
%%time
generate('What is the capital of England?',
         max_length=32000)

<think>
Okay, the user is asking for the capital of England. Let me start by recalling basic
geography facts. England is one of the four countries that make up the United Kingdom,
along with Scotland, Wales, and Northern Ireland. Now, I need to remember which of these
is the capital.

I know that London is a major city in England, and it's often referred to as the capital.
But wait, is it actually the capital? I should double-check to be sure. Sometimes people
might confuse the capital of the UK with the capital of England. The capital of the UK is
London, but the capital of England is also London. That makes sense because London is the
largest city in England and serves as the political, economic, and cultural hub.

Let me think if there's any other city that could be considered the capital of England. I
don't recall any other cities being referred to as the capital. For example, Edinburgh is
the capital of Scotland, Cardiff is the capital of Wales, and Belfast is the capital of
North

In [ ]:
%%time
generate('Can Geoffrey Hinton have a conversation with George Washington? Give the rationale before answering.',
         max_length=32000)

<think>
Okay, let's see. The question is whether Geoffrey Hinton can have a conversation with
George Washington. Hmm. First, I need to understand what the question is really asking.
It's not about whether they actually did converse, but rather if it's possible for them to
have such a conversation, given their respective times and circumstances.

So, Geoffrey Hinton is a Canadian-American cognitive scientist and machine learning
researcher. He's known for his work in deep learning and neural networks. He's been active
in the field for several decades, right? His work has been foundational in modern AI and
machine learning. He's still working and publishing, so he's definitely alive.

George Washington, on the other hand, was the first President of the United States,
serving from 1789 to 1797. He was a military leader during the American Revolutionary War.
He died in 1799, so he's been dead for over 225 years.

Now, the key here is the timeline. If we're talking about a conversation in t

## GSM8K

In [ ]:
generate('Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?',
         max_length=3200)

<think>
Okay, let's see. The cafeteria starts with 23 apples. They used 20 for lunch. So first, I
need to figure out how many apples are left after using 20. If they had 23 and used 20,
that means 23 minus 20. Let me do that calculation. 23 minus 20 is 3. So after lunch, they
have 3 apples left.

Then, they bought 6 more apples. So now, I need to add those 6 to the 3 apples they had
left. 3 plus 6 is 9. So, 3 plus 6 equals 9. Therefore, the cafeteria now has 9 apples. Let
me just check my steps again to make sure I didn't make a mistake. Start with 23, subtract
20 for lunch, which gives 3. Then add 6, which gives 9. Yeah, that seems right. I don't
think I missed anything here. The answer should be 9 apples.
</think>

1. **Initial Apples**: The cafeteria starts with 23 apples.
2. **Apples Used for Lunch**: They used 20 apples for lunch.
3. **Apples Left After Lunch**: Subtract the apples used from the initial amount:
   $23 - 20 = 3$ apples remaining.
4. **Apples Bought**: They bought 6

In [ ]:
generate("Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?",
         max_length=32000)

<think>
Okay, let's see. Weng earns $12 an hour for babysitting. She worked 50 minutes yesterday.
I need to figure out how much she earned. Hmm, so the problem is converting her time into
hours because her pay is per hour.

First, I remember that an hour is 60 minutes. So 50 minutes is less than an hour. To find
out how much time she actually worked in terms of hours, I should divide the minutes by
60. Let me write that down.

So, 50 minutes divided by 60 minutes per hour. That would be 50/60. Let me calculate that.
50 divided by 60 is equal to... Well, 50 divided by 60 is the same as 5/6 when simplified.
But maybe I should just do the division. 50 divided by 60 is 0.8333... repeating. So
approximately 0.8333 hours.

Now, since she earns $12 per hour, I need to multiply her hourly rate by the number of
hours she worked. So that's 12 dollars times 0.8333 hours. Let me do that multiplication.

12 times 0.8333... Hmm. Let me think. 12 times 0.8 is 9.6, and 12 times 0.0333 is
approximately

In [ ]:
generate("A deep-sea monster rises from the waters once every hundred years to feast on a ship and sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been built larger over time, so each new ship has twice as many people as the last ship. How many people were on the ship the monster ate in the first hundred years?",
         max_length=8000)

<think>
Okay, let's see. So the problem is about a deep-sea monster that comes up every hundred
years and eats a ship. Over three hundred years, it's eaten 847 people. The ships have
been getting bigger each time, with each new ship having twice as many people as the last
one. The question is asking how many people were on the first ship, the one in the first
hundred years.

Alright, let me break this down. The monster comes every hundred years, so in 300 years,
that's three times it's come up. Each time, the ship is larger than the previous one,
doubling in size each time. So the number of people on each ship is forming a geometric
sequence where each term is twice the previous term. The total number of people consumed
over the three centuries is 847. We need to find the number of people on the first ship.

Let me recall that a geometric sequence has a first term, let's call it a, and each
subsequent term is multiplied by a common ratio, which in this case is 2. So the sequence
would 

## Tool Use

In [ ]:

tools = [
    {
        "name": "get_weather",
        "description": "Get the weather in a city",
        "parameters": {"type": "object", "properties": {"city": {"type": "string", "description": "The city to get the weather for"}}}
    }
]

messages = [
    {
        "role": "user",
        "content": "Hello! How is the weather today in Copenhagen?"
    }
]

inputs = tokenizer.apply_chat_template(
    messages,
    enable_thinking=False, # True works as well, your choice!
    xml_tools=tools,
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt"
)

outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))


<|im_start|>system
## Metadata

Knowledge Cutoff Date: June 2025
Today Date: 09 July 2025
Reasoning Mode: /no_think

## Custom Instructions

You are a helpful AI assistant named SmolLM, trained by Hugging Face.

### Tools

You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags:

<tools>
{'name': 'get_weather', 'description': 'Get the weather in a city', 'parameters': {'type': 'object', 'properties': {'city': {'type':'string', 'description': 'The city to get the weather for'}}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call>

<|im_end|>
<|im_start|>user
Hello! How is the weather today in Copenhagen?<|im_end|>
<|im_start|>assistant
<think>

</think>
<tool_call>
{"name": "get_weather", "arguments": {"city": "Copenhagen"}}
</tool_call>


In [ ]:
tools = [
    {
        "name": "search_web",
        "description": "Allows you to search the internet to find useful and uptodate information",
        "parameters": {"type": "object", "properties": {"search_term": {"type": "string", "description": "what keywords to use for a search"}}}
    }
]

messages = [
    {
        "role": "user",
        "content": "When will OpenAI release their open weights model? What are the rumors"
    }
]

model_inputs = tokenizer.apply_chat_template(
    messages,
    enable_thinking=False, # True works as well, your choice!
    xml_tools=tools,
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt"
)

outputs = model.generate(model_inputs, max_new_tokens=32768)
print(tokenizer.decode(outputs[0]))

<|im_start|>system
## Metadata

Knowledge Cutoff Date: June 2025
Today Date: 09 July 2025
Reasoning Mode: /no_think

## Custom Instructions

You are a helpful AI assistant named SmolLM, trained by Hugging Face.

### Tools

You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags:

<tools>
{'name':'search_web', 'description': 'Allows you to search the internet to find useful and uptodate information', 'parameters': {'type': 'object', 'properties': {'search_term': {'type':'string', 'description': 'what keywords to use for a search'}}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call>

<|im_end|>
<|im_start|>user
When will OpenAI release their open weights model? What are the rumors<|im_end|>
<|im_start|>assistant
<think>

</think>
<tool_call>
{"

In [ ]:
tools = [
    {
        "name": "search_web",
        "description": "Allows you to search the internet to find useful and uptodate information",
        "parameters": {"type": "object", "properties": {"search_term": {"type": "string", "description": "what keywords to use for a search"}}}
    }
]

messages = [
    {
        "role": "user",
        "content": "Who won the nobel prize for chemistry in 2024"
    }
]

model_inputs = tokenizer.apply_chat_template(
    messages,
    enable_thinking=False, # True works as well, your choice!
    xml_tools=tools,
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt"
)

outputs = model.generate(model_inputs, max_new_tokens=32768)
print(tokenizer.decode(outputs[0]))

<|im_start|>system
## Metadata

Knowledge Cutoff Date: June 2025
Today Date: 09 July 2025
Reasoning Mode: /no_think

## Custom Instructions

You are a helpful AI assistant named SmolLM, trained by Hugging Face.

### Tools

You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags:

<tools>
{'name':'search_web', 'description': 'Allows you to search the internet to find useful and uptodate information', 'parameters': {'type': 'object', 'properties': {'search_term': {'type':'string', 'description': 'what keywords to use for a search'}}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call>

<|im_end|>
<|im_start|>user
Who won the nobel prize for chemistry in 2024<|im_end|>
<|im_start|>assistant
<think>

</think>
<tool_call>
{"name": "search_web", "arg

In [ ]:
tools = [
    {
        "name": "search_web",
        "description": "Allows you to search the internet to find useful and uptodate information",
        "parameters": {"type": "object", "properties": {"search_term": {"type": "string", "description": "what keywords to use for a search"}}}
    },
    {
        "name": "search_web",
        "description": "Allows you to search the internet to find useful and uptodate information",
        "parameters": {"type": "object", "properties": {"search_term": {"type": "string", "description": "what keywords to use for a search"}}}
    }
]

messages = [
    {
        "role": "user",
        "content": "What is a Labubu? and who made it?"
    }
]

model_inputs = tokenizer.apply_chat_template(
    messages,
    enable_thinking=False, # True works as well, your choice!
    xml_tools=tools,
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt"
)

outputs = model.generate(model_inputs, max_new_tokens=32768)
print(tokenizer.decode(outputs[0]))

<|im_start|>system
## Metadata

Knowledge Cutoff Date: June 2025
Today Date: 09 July 2025
Reasoning Mode: /no_think

## Custom Instructions

You are a helpful AI assistant named SmolLM, trained by Hugging Face.

### Tools

You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags:

<tools>
{'name':'search_web', 'description': 'Allows you to search the internet to find useful and uptodate information', 'parameters': {'type': 'object', 'properties': {'search_term': {'type':'string', 'description': 'what keywords to use for a search'}}}}
{'name':'search_web', 'description': 'Allows you to search the internet to find useful and uptodate information', 'parameters': {'type': 'object', 'properties': {'search_term': {'type':'string', 'description': 'what keywords to use for a search'}}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags

In [ ]:
tools = [
    {
        "name": "search_web",
        "description": "Allows you to search the internet to find useful and uptodate information",
        "parameters": {"type": "object", "properties": {"search_term": {"type": "string", "description": "what keywords to use for a search"}}}
    },
    {
        "name": "get_weather",
        "description": "Get the weather in a city",
        "parameters": {"type": "object", "properties": {"city": {"type": "string", "description": "The city to get the weather for"}}}
    }
]

messages = [
    {
        "role": "user",
        "content": "How do you feel?"
    }
]

model_inputs = tokenizer.apply_chat_template(
    messages,
    enable_thinking=False, # True works as well, your choice!
    xml_tools=tools,
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt"
)

outputs = model.generate(model_inputs, max_new_tokens=32768)
print(tokenizer.decode(outputs[0]))

<|im_start|>system
## Metadata

Knowledge Cutoff Date: June 2025
Today Date: 09 July 2025
Reasoning Mode: /no_think

## Custom Instructions

You are a helpful AI assistant named SmolLM, trained by Hugging Face.

### Tools

You may call one or more functions to assist with the user query.
You are provided with function signatures within <tools></tools> XML tags:

<tools>
{'name':'search_web', 'description': 'Allows you to search the internet to find useful and uptodate information', 'parameters': {'type': 'object', 'properties': {'search_term': {'type':'string', 'description': 'what keywords to use for a search'}}}}
{'name': 'get_weather', 'description': 'Get the weather in a city', 'parameters': {'type': 'object', 'properties': {'city': {'type':'string', 'description': 'The city to get the weather for'}}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <a